# 1. 位置合わせ

画像ごとに、カメラと被写体の位置関係が異なるので、マーカ4点の位置を合わせる。

In [3]:
!pip install -q pandas

import os, cv2, re, shutil
import numpy as np
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 6.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 9.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 9.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [4]:
#元画像があるDIR
IMG = '../data/cropped_imgs'

In [7]:
fnames = sorted(os.listdir(IMG))
fnames[:9]

['.DS_Store',
 'P_En1_05d_IMG_9379.png',
 'P_En1_06d_IMG_9388.png',
 'P_En1_07d_IMG_9397.png',
 'P_En1_08d_IMG_9406.png',
 'P_En1_09d_IMG_9415.png',
 'P_En1_10d_IMG_9424.png',
 'P_En1_11d_IMG_9433.png',
 'P_En1_12d_IMG_9442.png']

In [11]:
# !rm ../data/cropped_imgs/.DS_Store

In [5]:
#cv2.imshowで表示したウィンドウをクリックした時に座標を取得する関数
def onMouse(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        list_.append(x)
        list_.append(y)

In [6]:
list_all_imgs = []

In [11]:
#このセルを実行するとウィンドウが表示され、クリックでマーカの座標を変数に保存できる
#点を選び終わったらなんでも良いのでキーボードを押す
for fname in fnames:
    list_ = [fname]
    print(fname)
    img = cv2.imread(os.path.join(IMG,fname))
    window_name = fname
    img_ = cv2.resize(img, dsize=None, fx=2, fy=2)
    cv2.imshow(window_name, img_)
    cv2.setMouseCallback(window_name, onMouse)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    print(list_) # [file_name, x1, y1, ..., x4, y4]
    list_all_imgs.append(list_)

P_En2_05d_IMG_9380.png
['P_En2_05d_IMG_9380.png', 278, 185, 1896, 233, 250, 1730, 1836, 1763]
P_En2_06d_IMG_9389.png
['P_En2_06d_IMG_9389.png', 467, 299, 2049, 356, 418, 1829, 2011, 1880]
P_En2_07d_IMG_9398.png
['P_En2_07d_IMG_9398.png', 746, 367, 2376, 404, 706, 1911, 2325, 1978]
P_En2_08d_IMG_9407.png
['P_En2_08d_IMG_9407.png', 841, 407, 2457, 509, 743, 1943, 2362, 2080]
P_En2_09d_IMG_9416.png
['P_En2_09d_IMG_9416.png', 884, 328, 2513, 344, 860, 1870, 2482, 1921]
P_En2_10d_IMG_9425.png
['P_En2_10d_IMG_9425.png', 710, 720, 2302, 699, 702, 2250, 2346, 2279]
P_En2_11d_IMG_9434.png
['P_En2_11d_IMG_9434.png', 677, 643, 2267, 652, 630, 2164, 2261, 2233]
P_En2_12d_IMG_9443.png
['P_En2_12d_IMG_9443.png', 765, 650, 2342, 679, 716, 2165, 2333, 2239]
P_En2_13d_IMG_9452.png
['P_En2_13d_IMG_9452.png', 677, 652, 2269, 650, 667, 2174, 2280, 2210]
P_En3_05d_IMG_9381.png
['P_En3_05d_IMG_9381.png', 283, 155, 1900, 235, 231, 1684, 1815, 1743]
P_En3_06d_IMG_9390.png
['P_En3_06d_IMG_9390.png', 425, 321, 

In [12]:
list_all_imgs

[['P_En1_05d_IMG_9379.png', 489, 294, 2090, 282, 516, 1846, 2098, 1831],
 ['P_En1_06d_IMG_9388.png', 479, 289, 2068, 330, 443, 1836, 2035, 1878],
 ['P_En1_07d_IMG_9397.png', 709, 292, 2327, 314, 685, 1851, 2302, 1900],
 ['P_En1_08d_IMG_9406.png', 819, 279, 2444, 337, 754, 1837, 2370, 1929],
 ['P_En1_09d_IMG_9415.png', 900, 350, 2525, 439, 804, 1903, 2418, 2027],
 ['P_En1_10d_IMG_9424.png', 726, 708, 2316, 679, 718, 2257, 2359, 2279],
 ['P_En1_11d_IMG_9433.png', 692, 595, 2280, 613, 636, 2132, 2264, 2210],
 ['P_En1_12d_IMG_9442.png', 758, 605, 2337, 637, 703, 2130, 2313, 2211],
 ['P_En1_13d_IMG_9451.png', 684, 432, 2275, 447, 657, 1963, 2255, 2018],
 ['P_En2_05d_IMG_9380.png', 278, 185, 1896, 233, 250, 1730, 1836, 1763],
 ['P_En2_06d_IMG_9389.png', 467, 299, 2049, 356, 418, 1829, 2011, 1880],
 ['P_En2_07d_IMG_9398.png', 746, 367, 2376, 404, 706, 1911, 2325, 1978],
 ['P_En2_08d_IMG_9407.png', 841, 407, 2457, 509, 743, 1943, 2362, 2080],
 ['P_En2_09d_IMG_9416.png', 884, 328, 2513, 344, 86

In [24]:
# 座標をcsvに書き出し
df = pd.DataFrame(points_list)
df.columns = ['fname', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4']
df.to_csv('./seed_pack_2024_points_crop.csv', index=False)

In [7]:
# filenameでsort
df.sort_values('fname').to_csv('./seed_pack_2024_points_sorted.csv', index=False)
df_sorted = pd.read_csv('./seed_pack_2024_points_sorted.csv')
df_sorted

,fname,x1,y1,x2,y2,x3,y3,x4,y4
0,P_En1_05d_IMG_9379.png,489,294,2090,282,516,1846,2098,1831
1,P_En1_06d_IMG_9388.png,479,289,2068,330,443,1836,2035,1878
2,P_En1_07d_IMG_9397.png,709,292,2327,314,685,1851,2302,1900
3,P_En1_08d_IMG_9406.png,819,279,2444,337,754,1837,2370,1929
4,P_En1_09d_IMG_9415.png,900,350,2525,439,804,1903,2418,2027
...,...,...,...,...,...,...,...,...,...
76,P_Wi3_09d_IMG_9423.png,888,412,2512,344,898,2005,2524,2013
77,P_Wi3_10d_IMG_9432.png,759,704,2350,623,768,2288,2413,2297
78,P_Wi3_11d_IMG_9441.png,714,701,2303,684,652,2269,2286,2354
79,P_Wi3_12d_IMG_9450.png,854,686,2430,754,716,2240,2337,2401


In [48]:
# SEG = '' #セグメンテーション画像の格納DIR
DST = '../data/seed_pack2024_resized_seg_transformed' #変換後の画像の格納DIR

## ポイントの位置を確認

In [20]:
import matplotlib.pyplot as plt
for idx, row in df.iterrows():
    fname = row[0]
    img = cv2.imread(os.path.join(IMG, fname))
    coord = [[row[1]//2, row[2]//2], [row[3]//2, row[4]//2], [row[5]//2, row[6]//2], [row[7]//2, row[8]//2]]
    
    for center in coord:
        cv2.circle(img, center, 3, (255, 0, 0), thickness=3)
    try:
        cv2.imwrite(f'../data/check_points/{fname}', img)
    except:
        print(fname)
#     break
    

***

## 射映変換

In [12]:
# SEG = '../data/cropped_imgs'
# DST = '../data/cropped_imgs_transformed'

SEG = '../data/cropped_segs'  # 変換前
DST = '../data/cropped_segs_transformed2'  # 変換後

In [14]:
# 基準座標4点を指定
base_coord = [[700,300], [2100, 300], [700, 1800], [2100, 1800]]

for idx, row in df.iterrows():
    fname = row[0]
    img = cv2.imread(os.path.join(SEG, fname))
    # half sizeで座標を取ったのでxy座標を2倍
    coord = [[row[1], row[2]], [row[3], row[4]], [row[5], row[6]], [row[7], row[8]]]
    M = cv2.getPerspectiveTransform(np.float32(coord), np.float32(base_coord))
    dst = cv2.warpPerspective(img,M,(img.shape[1], img.shape[0]))
    cv2.imwrite(os.path.join(DST, fname), dst)

In [36]:
df

,fname,x1,y1,x2,y2,x3,y3,x4,y4
0,P_En1_05d_IMG_9379.png,489,294,2090,282,516,1846,2098,1831
1,P_En1_06d_IMG_9388.png,479,289,2068,330,443,1836,2035,1878
2,P_En1_07d_IMG_9397.png,709,292,2327,314,685,1851,2302,1900
3,P_En1_08d_IMG_9406.png,819,279,2444,337,754,1837,2370,1929
4,P_En1_09d_IMG_9415.png,900,350,2525,439,804,1903,2418,2027
...,...,...,...,...,...,...,...,...,...
76,P_Wi3_05d_IMG_9387.png,222,264,1827,263,218,1860,1787,1869
77,P_Wi3_06d_IMG_9396.png,392,316,1975,304,365,1905,1959,1921
78,P_Wi3_07d_IMG_9405.png,819,511,2439,497,779,2102,2404,2153
79,P_Wi3_08d_IMG_9414.png,877,338,2501,367,793,1926,2413,2026


In [46]:
SEG = '../data/cropped_imgs'
DST = '../data/cropped_imgs_transformed'


previous = ''
for idx, row in df.iterrows():
    fname = row[0]
    img = cv2.imread(os.path.join(SEG, fname))
    root_name = re.findall('(.+_.+)_.+_.+_.+', fname)[0] # xxx_日付.pngからxxxだけ抜き出し
    if root_name == previous:
        # half sizeで座標を取ったのでxy座標を2倍
        coord = [[row[1], row[2]], [row[3], row[4]], [row[5], row[6]], [row[7], row[8]]]
        print(coord)
        M = cv2.getPerspectiveTransform(np.float32(coord), np.float32(base_coord))
        dst = cv2.warpPerspective(img,M,(img.shape[1], img.shape[0]))
        cv2.imwrite(os.path.join(DST, fname), dst)
    else:
        print(root_name)
        cv2.imwrite(os.path.join(DST, fname), img)
        #変換対象の座標
        base_coord = [[row[1], row[2]], [row[3], row[4]], [row[5], row[6]], [row[7], row[8]]]
    previous = root_name


P_En1
[[479, 289], [2068, 330], [443, 1836], [2035, 1878]]
[[709, 292], [2327, 314], [685, 1851], [2302, 1900]]
[[819, 279], [2444, 337], [754, 1837], [2370, 1929]]
[[900, 350], [2525, 439], [804, 1903], [2418, 2027]]
[[726, 708], [2316, 679], [718, 2257], [2359, 2279]]
[[692, 595], [2280, 613], [636, 2132], [2264, 2210]]
[[758, 605], [2337, 637], [703, 2130], [2313, 2211]]
[[684, 432], [2275, 447], [657, 1963], [2255, 2018]]
P_En2
[[467, 299], [2049, 356], [418, 1829], [2011, 1880]]
[[746, 367], [2376, 404], [706, 1911], [2325, 1978]]
[[841, 407], [2457, 509], [743, 1943], [2362, 2080]]
[[884, 328], [2513, 344], [860, 1870], [2482, 1921]]
[[710, 720], [2302, 699], [702, 2250], [2346, 2279]]
[[677, 643], [2267, 652], [630, 2164], [2261, 2233]]
[[765, 650], [2342, 679], [716, 2165], [2333, 2239]]
[[677, 652], [2269, 650], [667, 2174], [2280, 2210]]
P_En3
[[425, 321], [2013, 376], [379, 1831], [1968, 1885]]
[[742, 373], [2369, 435], [673, 1890], [2296, 1983]]
[[817, 315], [2443, 428], [6

In [57]:
from glob import glob
import matplotlib.pyplot as plt
import numpy as np

IMG = '../data/cropped_imgs_transformed/*'

path_imgs = sorted(glob(IMG))

for path in path_imgs:
    img = cv2.imread(path)
    seg = cv2.imread(path.replace('cropped_imgs_transformed', 'cropped_segs_noise_removed'))
    seg[:, :, :2] = 0
    img_with_seg = (img * 0.8 + seg * 0.2).astype(np.uint8)
    cv2.imwrite(path.replace('cropped_imgs_transformed', 'img_with_seg'), img_with_seg)
#     plt.imshow(img_with_seg.astype(np.uint8))
#     plt.show()
#     break

## 参考

[Pythonで画像の座標を取得する方法を現役エンジニアが解説【初心者向け】](https://techacademy.jp/magazine/51035)

[pandas.DataFrameのforループ処理（イテレーション）](https://note.nkmk.me/python-pandas-dataframe-for-iteration/)

[画像の幾何学変換](http://opencv.jp/opencv-2.1/cpp/geometric_image_transformations.html)

[Python, OpenCVで幾何変換（アフィン変換・射影変換など）](https://note.nkmk.me/python-opencv-warp-affine-perspective/)